<a href="https://colab.research.google.com/github/lucas-fpaiva/AnalisePred_I/blob/main/Projeto/car_eval/classifier_car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Avaliar se o preço de compra do veículo é aceitável


In [2]:
import pandas as pd
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import seaborn as sns

from scipy.io.arff import loadarff 
from scipy.io import arff

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import precision_score as ps
from sklearn.metrics import recall_score as rs
from sklearn.metrics import f1_score as f1
from sklearn.metrics import accuracy_score as acs
from sklearn.metrics import auc, roc_auc_score


from sklearn.metrics import plot_confusion_matrix

from sklearn.datasets import make_classification
from ast import literal_eval

import warnings
warnings.filterwarnings('ignore')


In [3]:
def grid_best_model (model="DT", names=False, grid=""):
  #A função retorna o modelo com os melhores parametros, os melhores parametros e o melhor score
  scoring = 'accuracy'

  if names == True:
    print("LR: LinearRegression\nKNN: KNeighborsClassifier\nMLPClassifier\nDT: DecisionTreeClassifier\nRF: RandomForestClassifier",
          "\nSVM: SupportVectorMachineClassifier\nBNB: BernoulliNaiveBayes\nADA: AdaBoostClassifier\nGB: GradientBoostingClassifier\n")
    
  if model == "DT":
    gs_cv = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=grid['DT'], cv= 5, scoring=scoring, n_jobs=-1)
    gs_cv.fit(x_treino, y_treino)
    return DecisionTreeClassifier(**gs_cv.best_params_).fit(x_treino, y_treino), gs_cv.best_params_
      
  if model=="MLP":
    gs_cv = GridSearchCV(estimator=MLPClassifier(), param_grid=grid['MLP'], cv= 5, scoring=scoring, n_jobs=-1)
    gs_cv.fit(x_treino, y_treino)
    return MLPClassifier(**gs_cv.best_params_), gs_cv.best_params_
    
  if model == "RF":
    gs_cv = GridSearchCV(estimator=RandomForestClassifier(), param_grid=grid['RF'], cv= 5, scoring=scoring, n_jobs=-1)
    gs_cv.fit(x_treino, y_treino)
    return RandomForestClassifier(**gs_cv.best_params_), gs_cv.best_params_

  if model == "KNN":
    gs_cv = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=grid['KNN'], cv= 5, scoring=scoring, n_jobs=-1)
    gs_cv.fit(x_treino, y_treino)
    return KNeighborsClassifier(**gs_cv.best_params_), gs_cv.best_params_

  if model == "LR":
    gs_cv = GridSearchCV(estimator=LogisticRegression(), param_grid=grid['LR'], cv= 5, scoring=scoring, n_jobs=-1)
    gs_cv.fit(x_treino, y_treino)
    return LogisticRegression(**gs_cv.best_params_), gs_cv.best_params_

  if model == "BNB":
    gs_cv = GridSearchCV(estimator=BernoulliNB(), param_grid=grid['BNB'], cv= 5, scoring=scoring, n_jobs=-1)
    gs_cv.fit(x_treino, y_treino)
    return BernoulliNB(**gs_cv.best_params_).fit(x_treino, y_treino), gs_cv.best_params_

  if model == "SVM":
    gs_cv = GridSearchCV(estimator=SVC(), param_grid=grid['SVM'], cv= 5, scoring=scoring, n_jobs=-1)
    gs_cv.fit(x_treino, y_treino)
    return SVC(**gs_cv.best_params_).fit(x_treino, y_treino), gs_cv.best_params_

  if model == "GBC":
    gs_cv = GridSearchCV(estimator=GradientBoostingClassifier(), param_grid=grid['GBC'], cv= 5, scoring=scoring, n_jobs=-1)
    gs_cv.fit(x_treino, y_treino)
    return GradientBoostingClassifier(**gs_cv.best_params_).fit(x_treino, y_treino), gs_cv.best_params_
        
  return print("LR: LinearRegression\nKNN: KNeighborsClassifier\nMLPClassifier\nDT: DecisionTreeClassifier\nRF: RandomForestClassifier",
              "\nSVM: SupportVectorMachineClassifier\nBNB: BernoulliNaiveBayes\nADA: AdaBoostClassifier\nGB: GradientBoostingClassifier\n")
  
#Parâmetros do Grid
grid = {
    'RF':{ #based on: Discrete Simulation Optimization for Tuning Machine Learning Method Hyperparameters
        'n_estimators': [10, 100],
        'max_depth' : range(5, 50, 5),
        'min_samples_split' : [2, 11],
        'min_samples_leaf' : [1, 11],
        #'max_features' : [1,54],
        'criterion' :['gini', 'entropy'],
        'random_state' : [0]},

    'MLP':{ #based on: Discrete Simulation Optimization for Tuning Machine Learning Method Hyperparameters
        'hidden_layer_sizes': [3, 5, 8, 12, 15, 20, 25, 30, 50, 80],
        'learning_rate' : ['constant','adaptive'],
        'learning_rate_init' : [0.0005, 0.001, 0.01, 0.05, 0.1],
        'max_iter': [200],
        'activation' : ['relu', 'logistic', 'tanh'],
        'solver' : ['adam', 'sgd'],
        'random_state' : [0]},

    'DT':{ #based on: Random Forest
        'max_depth' : range(5, 50, 5),
        'min_samples_split' : [2, 11],
        'min_samples_leaf' : [1, 11],
        #'max_features' : [1,54],
        'criterion' :['gini', 'entropy'],
        'random_state' : [0]},

    'KNN':{ #based on:
        'n_neighbors': range(5, 50, 5),
        'weights': ['uniform', 'distance'],
        'metric' : ['manhattan', 'euclidean']},

    'LR':{ #based on: Logistic Regression Hyperparameter Optimization for Cancer Classification
        'penalty' : ['l1', 'l2', 'elasticnet'],
        'l1_ratio' : [0, 1],
        #'learning_rate' : ['constant', 'optimal', 'invscaling', 'adaptive'],
        'eta0' : [0, 2],
        'tol' : [0.00001, 0.01], #Alpha
        'warm_start' : {True, False},
        'random_state' : [0]},

    'BNB':{ #based on:
        'alpha' : np.arange(0.0, 1.0, 0.05)},
    
    'SVM':{ #based on:
        'C' : [0.01, 0.1, 1, 10, 100, 1000],
        'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
        'gamma' : ['auto', 'scale'], #[0.001, 0.01, 0.1, 0.5, 1, 10, 100],
        'probability' : [True],
        'random_state' : [0]},
    
    'GBC':{ #based on:
        'n_estimators': [100],
        'max_depth' : [2, 3, 5, 7, 10],
        'learning_rate' : [0.025, 0.05, 0.1, 0.2, 0.3],
        'min_samples_split' : [2, 5, 10, 20],
        'max_features' : ['log2', 'sqrt', 0.25, 1.0],
        'subsample' : [0.15, 0.5, 0.75, 1.0],
        'random_state' : [0]}
}

In [4]:
def metrics (y_true, y_pred, y_pred_prob):
  return [acs(y_true, y_pred), ps(y_true, y_pred, average = 'weighted'), f1(y_true, y_pred, average = 'weighted'),
          rs(y_true, y_pred, average = 'weighted'), roc_auc_score(y_true, y_pred_prob, average = 'weighted', multi_class = 'ovr')]

## Lendo o Dataset

In [5]:
data = pd.read_csv('https://raw.githubusercontent.com/lucas-fpaiva/AnalisePred_I/main/Projeto/car_eval/data_processed.csv', index_col=0)
features = data.columns[1:-1]

In [6]:
data.head()

,class,buying_price_vhigh,buying_price_high,buying_price_med,buying_price_low,maintenance_price_vhigh,maintenance_price_high,maintenance_price_med,maintenance_price_low,doors_2,...,persons_4,persons_more,luggage_boot_size_small,luggage_boot_size_med,luggage_boot_size_big,safety_low,safety_med,safety_high,id,fold
0,0,1,0,0,0,1,0,0,0,1,...,0,0,1,0,0,1,0,0,0,5
1,0,1,0,0,0,1,0,0,0,1,...,0,0,1,0,0,0,1,0,1,5
2,0,1,0,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,1,2,5
3,0,1,0,0,0,1,0,0,0,1,...,0,0,0,1,0,1,0,0,3,4
4,0,1,0,0,0,1,0,0,0,1,...,0,0,0,1,0,0,1,0,4,2


## GridSearch dos Modelos

In [15]:
data_results = pd.DataFrame([],columns=['accuracy', 'precision', 'f1_score','recall', 'auc', 'fold','model','parameters'])
models = ["LR", "KNN", "MLP", "DT", "RF", "BNB", "SVM", "GBC"]

for name_model in models:
  results = []
  for k in range(1,6):
    treino = data[data.fold!=k]
    teste  = data[data.fold==k]
    
    x_treino = treino[features]
    y_treino = treino['class']

    x_teste = teste[features]
    y_teste = teste['class']

    model, best_parameters = grid_best_model(model=name_model, grid=grid)
    model.fit(x_treino, y_treino)
    y_pred= model.predict(x_teste)
    y_pred_prob = model.predict_proba(x_teste)

    print("Modelo:", name_model)
    print("Melhor parametro:", best_parameters)         
    print("Fold", k, metrics(y_teste.values, y_pred, y_pred_prob))

    plt.figure();
    plot_confusion_matrix(model, x_teste, y_teste, normalize=True);
    plt.title(name_model+'_f'+ str(k));
    plt.savefig('/content/drive/MyDrive/AnalisePred_I/confusion_matrix/'+ name_model+'_f'+ str(k) +'.pdf', format="pdf", dpi=1000, bbox_inches="tight");
    plt.close();

    result = metrics(y_teste, y_pred, y_pred_prob)
    result.append(k)
    result.append(name_model)
    result.append(best_parameters)
    
    results.append(result)
        
  data_results = pd.concat([data_results, pd.DataFrame(results,columns=['accuracy', 'precision', 'f1_score','recall', 'auc', 'fold','model','parameters'])])
  data_results.to_csv('/content/drive/MyDrive/AnalisePred_I/Car_results_'+ name_model + '.csv')


KeyboardInterrupt: ignored